In [1]:
import pandas as pd
import numpy as np
import warnings
import pyodbc
import random
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from collections import Counter
import multiprocessing as mp
import ray
import datetime

warnings.filterwarnings(action = 'ignore')

2023-12-06 08:12:01,962	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.3 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


#### 유전 알고리즘 활용한 페어 트레이딩_섹터 확장_v1

* v8
 - 비중 제한 없애기
 - child 생성 시 볼린저 밴드 계산 기간과 비중 모두 BLX-alpha crossover 적용
 - mutation 시 랜덤넘버 생성을 통해 각 파라미터에 더하기
 - 변이 이후 수선 기능 추가. 1) 롱/숏 비중 각 1로 맞추기
 - train set 기간 0.7에 대해 시뮬레이션 해보기
 - 정상성 테스트 제외: rolling base 기반 표준화를 사용하기 때문

In [2]:
conn_pcor = pyodbc.connect('driver={Oracle in OraClient18Home1};dbq=PCOR;uid=EF0SEL;pwd=EF0SEL#076')
conn_quant = pyodbc.connect('driver={SQL Server};server=46.2.90.172;database=quant;uid=index;pwd=samsung@00')
conn_wisefn = pyodbc.connect('driver={SQL Server};server=46.2.90.172;database=wisefn;uid=index;pwd=samsung@00')

In [3]:
os.chdir("..")
os.chdir("..")

os.chdir('00_data')

#df_const = pd.read_json('230927_index_constituent.json')
df_prc_raw = pd.read_json('230927_stk_prc_daily.json')
#df_turnover = pd.read_json('230927_stk_turnover.json')
#df_sector = pd.read_json('230927_stk_sector.json')
df_mktcap = pd.read_json('230927_stk_mktcap.json')
#df_turnover_daily = pd.read_json('231011_stk_turnover_daily.json')

os.chdir("..")
os.chdir('02_Trading Strategies')
os.chdir('231026_유전 알고리즘을 활용한 페어트레이딩')

In [4]:
sector_code = 'IKS013'

In [5]:
sql_dt = f'''
        SELECT TRD_DT, CLOSE_PRC
        FROM TS_IDX_DAILY
        WHERE 1=1
        AND SEC_CD = '{sector_code}'
        AND TRD_DT > '20091231'
        ORDER BY TRD_DT ASC
'''

df_dt = pd.read_sql(sql_dt, conn_wisefn)

In [6]:
df_dt['TRD_DT'] =  pd.to_datetime(df_dt['TRD_DT'])
df_dt['M'] = df_dt['TRD_DT'].dt.to_period('M')
dt_m = [max(df_dt[df_dt['M']==p]['TRD_DT']) for p in df_dt['M'].unique()]
dt_m = [int(x.strftime('%Y%m%d')) for x in dt_m]

In [7]:
def objective_func(df, params):
    
    #stk_list = sorted(list(set(df['COMP_CD'])))
    
    params_period = params[0]
    params_init = params[1]
    params_exit = params[2]
    params_wt = params[3:]

    df_sp = np.log(df) * params_wt
    df_sp = df_sp.sum(axis=1)
    
    df_sp_norm = (df_sp - df_sp.rolling(params_period).mean())/df_sp.rolling(params_period).std()
    df_sp_norm.dropna(axis=0, inplace=True)
    
    init_dt = []
    exit_dt = []
    init_yn = 0
    for i in range(len(df_sp_norm)):
        if init_yn == 0:
            if df_sp_norm.iloc[i] < params_init:
                init_dt.append(df_sp_norm.index[i])
                init_yn = 1
        
        elif init_yn == 1:
            if df_sp_norm.iloc[i] > params_exit:
                exit_dt.append(df_sp_norm.index[i])
                init_yn = 0
        
    if len(init_dt) > len(exit_dt):
        if len(init_dt) - len(exit_dt) > 1:
            print("error")
        else:
            init_dt = init_dt[:-1]
    
    ret_list = []
    cum_ret = 1
    for i in range(len(init_dt)):
        dt_1 = init_dt[i]
        dt_2 = exit_dt[i]
        
        prc_1 = df.loc[dt_1]
        prc_2 = df.loc[dt_2]
        
        ret = prc_2/prc_1 - 1
        ret = (ret * params_wt).sum()
        ret_list.append(ret)
        cum_ret *= (1+ret)
    
    return cum_ret

In [27]:
def generate_weight(length):
    
    pos_len = random.randint(1,length - 1)
    neg_len = length - pos_len
    
    pos_wt = []
    for i in range(pos_len):
        p = random.uniform(0,1)
        pos_wt.append(p)
    
    pos_wt = [x/sum(pos_wt) for x in pos_wt]
    
    neg_wt = []
    for i in range(neg_len):
        n = random.uniform(0,1)
        neg_wt.append(n)
    
    neg_wt = [-x/sum(neg_wt) for x in neg_wt]
    
    wt = pos_wt + neg_wt
    
    random.shuffle(wt)
    
    return wt  
        

def generate_params(df, length):
    
    params_period = random.randint(20,20*24)
    params_init = random.uniform(-4,0)
    params_exit = random.uniform(0,4)
    params_wt = generate_weight(length)
    
    params = [params_period, params_init, params_exit, *params_wt]
            
    return params
    

In [28]:
def generate_pop(df, size, length):
    population = []
    
    for i in range(size):
        
        params = generate_params(df,length)
        population.append(params)
        
    return population

In [29]:
def compute_performance(df, population):
    
    obj_list = []
    for individual in population:
        obj = objective_func(df, individual)
        obj_list.append([individual,obj])
    
    pop_sorted = sorted(obj_list, key=lambda x: x[1], reverse=True)
    
    return pop_sorted

In [30]:
def select_survivors(population_sorted, best_sample, lucky_few, length, df):
    
    next_gen = []
    
    for i in range(best_sample):
        if population_sorted[i][1] > 0:
            next_gen.append(population_sorted[i][0])
    
    lucky_index = np.random.choice(list(range(len(population_sorted))), lucky_few, replace=False)
    for i in lucky_index:
        next_gen.append(population_sorted[i][0])
    
    next_gen_2 = []
    for item in next_gen:
        if item not in next_gen_2:
            next_gen_2.append(item)
    
    
    while len(next_gen_2) < best_sample + lucky_few:
        next_gen_2.append(generate_params(df, length))
        
    random.shuffle(next_gen_2)
    
    return next_gen_2

In [31]:
def create_child(param1, param2, alpha):
    
    child = param1.copy()
    
    for i in range(len(child)):
        
        dist = abs(param1[i]-param2[i])
        
        l = min(param1[i],param2[i]) - alpha * dist
        u = max(param1[i],param2[i]) + alpha * dist

        child[i] = l + random.random() * (u-l)
    
    child[0] = int(np.round(child[0]))
    child[0] = min(20*24, child[0])
    child[0] = max(20, child[0])
    
    return child


def create_children(df, population, n_child, alpha, r_param):
    
    k = 0
    children = []

    fitness = [objective_func(df, x) for x in population]
    roulette = [x - min(fitness) + (max(fitness) - min(fitness))/(r_param - 1) for x in fitness]
    #fitness_2 = [x if x >= 0 else 0 for x in fitness]
    
    prob = [x/sum(roulette) for x in roulette]
    
    while k < n_child:
        try:
            sample = np.random.choice(list(range(len(prob))), 2, replace=True, p = prob)
        except:
            sample = np.random.choice(list(range(len(prob))), 2, replace=True)
        parents = [population[sample[0]], population[sample[1]]]
        
        child = create_child(parents[0], parents[1], alpha)
        children.append(child)
        
        k += 1

    return children

In [32]:
def mutation1(param, prob):
    
    r1 = random.random()
    
    param_mutate = param.copy()
    
    if r1 < prob:
        m1 = np.random.normal(0, 30)
        new_period = m1 + param[0]
        new_period = max(20, new_period)
        new_period = min(20*24, new_period)
        
        param_mutate[0] = int(np.round(new_period))
        
        m2 = np.random.normal(0,0.02,len(param) - 1)
        param_mutate[1:] = [x+y for (x,y) in zip(param_mutate[1:], m2)]
    
    return param_mutate


################################################
def mutate_pop(population, prob1):
    
    for i in range(len(population)):
        population[i] = mutation1(population[i], prob1)
    
    return population

In [33]:
def fix_1(population, df, length):
    
    # 수선: 비중 조절
    
    fixed_pop = []
    
    for child in population:
    
        child_wt = child[3:]
        child_pos = [x if x >= 0 else 0 for x in child_wt]
        child_neg = [x if x < 0 else 0 for x in child_wt]

        try:
            child_pos_adj = [x/sum(child_pos) for x in child_pos]
            child_neg_adj = [-x/sum(child_neg) for x in child_neg]
            child_wt_adj = [x+y for (x,y) in zip(child_pos_adj, child_neg_adj)]
            adj_child = child[:3] + child_wt_adj
        
        except:
            new_wt = generate_params(df, length)[3:]
            adj_child = child[:3] + new_wt
        
        fixed_pop.append(adj_child)
    
    return fixed_pop

In [34]:
def simulation_func(df, params):
    
    #stk_list = sorted(list(set(df['COMP_CD'])))
    
    params_period = params[0]
    params_init = params[1]
    params_exit = params[2]
    params_wt = params[3:]

    df_sp = np.log(df) * params_wt
    df_sp = df_sp.sum(axis=1)
    
    df_sp_norm = (df_sp - df_sp.rolling(params_period).mean())/df_sp.rolling(params_period).std()
    df_sp_norm.dropna(axis=0, inplace=True)
    
    init_dt = []
    exit_dt = []
    init_yn = 0
    for i in range(len(df_sp_norm)):
        if init_yn == 0:
            if df_sp_norm.iloc[i] < params_init:
                init_dt.append(df_sp_norm.index[i])
                init_yn = 1
        
        elif init_yn == 1:
            if df_sp_norm.iloc[i] > params_exit:
                exit_dt.append(df_sp_norm.index[i])
                init_yn = 0
        
    if len(init_dt) > len(exit_dt):
        if len(init_dt) - len(exit_dt) > 1:
            print("error")
        else:
            init_dt = init_dt[:-1]
    
    ret_list = []
    long_ret_list = []
    short_ret_list = []
    cum_ret = 1
    for i in range(len(init_dt)):
        dt_1 = init_dt[i]
        dt_2 = exit_dt[i]
        
        prc_1 = df.loc[dt_1]
        prc_2 = df.loc[dt_2]

        
        ret = prc_2/prc_1 - 1
        ret_ls = (ret * params_wt).sum()
        ret_list.append(ret_ls)        
        cum_ret *= (1+ret_ls)
        
        long_wt = [x if x >= 0 else 0 for x in params_wt]
        long_ret = (ret * long_wt).sum()
        long_ret_list.append(long_ret)
        
        short_wt = [x if x < 0 else 0 for x in params_wt]
        short_ret = (ret * short_wt).sum()
        short_ret_list.append(short_ret)
        
        
    
    return cum_ret, init_dt, exit_dt, ret_list, long_ret_list, short_ret_list

In [35]:
sector_code_list = ['IKS005', 'IKS006', 'IKS007', 'IKS008', 'IKS009', 'IKS010', 'IKS011', 'IKS012', 
                    'IKS013', 'IKS014', 'IKS015', 'IKS016', 'IKS017', 'IKS018', 'IKS019', 'IKS020', 
                    'IKS024', 'IKS025', 'IKS026']

In [36]:
sector_name_list = ['음식료품', '섬유의복', '종이목재', '화학', '의약품', '비금속광물', '철강금속', 
                    '기계', '전기전자', '의료정밀', '운수장비', '유통업', '전기가스업', '건설업', 
                    '운수창고업', '통신업', '증권', '보험업', '서비스업']

In [37]:
dict_sector_name = dict(zip(sector_code_list, sector_name_list))

In [38]:
i = 0.70
cap_hurdle = 200 * 1000000000
max_stk_cnt = 10

base_d = dt_m[int(np.round(len(dt_m)*i,0))]
train_begin_dt = dt_m[0]
train_end_dt = base_d

cap_filter = df_mktcap.copy()
cap_filter = cap_filter[cap_filter['BASE_D'] == base_d]
cap_filter = cap_filter[cap_filter['MKTCAP'] > cap_hurdle]
cap_filter.sort_values('MKTCAP', ascending = False, inplace = True)

dict_train = {}
dict_test = {}
dict_stk = {}

for sector_code in sector_code_list:
    
    sql_const = f'''
                SELECT TRD_DT, CONCAT('A',STK_CD) COMP_CD, STK_NM_KOR COMP_NM
                FROM TS_STK_ISSUE
                WHERE 1=1
                AND KS200_TYP = 1
                AND TRD_DT = '{base_d}'
                AND KSC_CD = '{sector_code}'
    '''

    df_const = pd.read_sql(sql_const, conn_wisefn)

    df_train = df_prc_raw.copy()
    df_train = df_train[df_train['TRD_DT'] >= train_begin_dt]
    df_train = df_train[df_train['TRD_DT'] <= train_end_dt]
    df_train = df_train[df_train['COMP_CD'].isin(df_const['COMP_CD'])]
    df_train = df_train.sort_values(by = ['TRD_DT','COMP_CD'])
    #stk_list = sorted(list(set(df_train['COMP_CD'])))


    df_train = df_train.pivot(index = 'TRD_DT', columns = 'COMP_CD')
    df_train = df_train.droplevel(axis = 1, level = 0)
    #df_train = df_train[stk_list]
    df_train.dropna(axis=1, inplace=True)
    stk_list = sorted(list(df_train.columns))
    
    # 시가총액 필터링
    stk_list = cap_filter[cap_filter['COMP_CD'].isin(stk_list)]['COMP_CD'][:max_stk_cnt].to_list()

    df_train = df_train[stk_list]
    
    df_test = df_prc_raw.copy()
    df_test = df_test[df_test['TRD_DT'] >= train_end_dt]
    #df_test = df_prc[df_prc['TRD_DT'] <= train_end_dt]
    df_test = df_test[df_test['COMP_CD'].isin(df_const['COMP_CD'])]
    df_test = df_test.sort_values(by = ['TRD_DT','COMP_CD'])

    df_test = df_test.pivot(index = 'TRD_DT', columns = 'COMP_CD')
    df_test = df_test.droplevel(axis = 1, level = 0)
    df_test = df_test[stk_list]
    
    dict_train[sector_code] = df_train
    dict_test[sector_code] = df_test
    dict_stk[sector_code] = stk_list

In [53]:
mp.cpu_count()

16

2023-12-06 10:43:07,922	INFO worker.py:1664 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.8.8
Ray version:,2.8.0
Dashboard:,http://127.0.0.1:8265


In [39]:
optimized_params = {}

for sector_code in sector_code_list:
    
    print(f'섹터: {dict_sector_name[sector_code]}')
    print(' ')
    
    df_train = dict_train[sector_code]
    df_test = dict_test[sector_code]
    stk_list = dict_stk[sector_code]
    
    if len(stk_list) == 0:
        continue
    
    
    print(f'  Train set: {train_begin_dt} ~ {train_end_dt}')
    print(f'  Test set: {train_end_dt} ~ {max(df_test.index)}')
    print(' ')
    print(f'  유전 알고리즘 시작')
    n_generation = 200
    population = 1000
    best_sample = int(population/4)
    lucky_few = int(population/4)
    prob1 = 0.01
    #prob2 = 0.01
    alpha = 0.10
    r_param = 4
    param_length = len(stk_list)

    pop = generate_pop(df_train, size = population, length = param_length)

    best_gene = []
    best_perf = []

    g = 0

    while True:

        pop_sorted = compute_performance(df_train, pop)

        survivors = select_survivors(pop_sorted, best_sample, lucky_few, param_length, df_train)

        children = create_children(df_train, survivors, population, alpha, r_param)

        new_generation = mutate_pop(children, prob1)

        if len(new_generation) < population:
            #print(f'  세대수 부족: {len(new_generation)}')
            break

        fixed_generation = fix_1(new_generation, df_train, param_length)

        pop = fixed_generation
        best_gene.append(pop_sorted[0][0])
        best_perf.append(pop_sorted[0][1])


        print(f'====== {g}th generation ends ======')
        print(pop_sorted[0])


        pop_wt = [x[0][3:] for x in pop_sorted]
        dist_wt = [sum([(x-y)**2 for (x,y) in zip(pop_wt[0],z)]) for z in pop_wt]
        hurdle = 0.025
        dist_wt_hurdle = [1 for x in dist_wt if x < hurdle]

        print(f'    최다 출현 개체수: {sum(dist_wt_hurdle)}')
        if sum(dist_wt_hurdle) > population * 0.7:
            break

        g += 1

        if g > n_generation:
            break

        print(' ')

    print(f'  유전 알고리즘 종료')
    print(' ')
    
    optimized_params[sector_code] = pop_sorted[0][0]

    

    

섹터: 음식료품
 
  Train set: 20100129 ~ 20191129
  Test set: 20191129 ~ 20230927
 
  유전 알고리즘 시작
====== 0th generation ends ======
[[38, -2.332793077567054, 3.6552020542152093, 0.004048418622554776, 0.6056526430176735, -0.049824036974887534, 0.036443606813192676, -0.14433042173603947, -0.026834891484390378, -0.3255009950026487, 0.31462435225464963, 0.03923097929192942, -0.45350965480203387], 11.583488234556828]
    최다 출현 개체수: 1
 
====== 1th generation ends ======
[[409, -0.8535618689723088, 3.3035267390742087, -0.07372638171098801, 0.4431911126287841, -0.03605267432756199, -0.15042809518717232, 0.16870633535812213, -0.5138713917906448, -0.05296335421021835, 0.3443916234129604, -0.17295810277341447, 0.04371092860013343], 8.33186266162136]
    최다 출현 개체수: 1
 
====== 2th generation ends ======
[[119, -0.9657810323784586, 3.206865221378426, -0.40276395767194545, 0.10046670132501333, 0.007480024022269133, 0.07336469679764215, -0.03578993849637611, -0.022236573443724757, 0.08608515871733265, 0.5790

====== 25th generation ends ======
[[270, -1.5728406565003528, 2.752489781802422, -0.0030471116148265576, -0.21771579587269255, -0.12150632291425813, -0.08313546153064123, -0.13994953895642578, -0.3451615503707671, 0.12102112963355648, 0.8789788703664436, -0.04238688614479938, -0.04709733259558936], 26.942971194637874]
    최다 출현 개체수: 9
 
====== 26th generation ends ======
[[281, -1.5925419762185538, 2.725506241164937, -0.036587301744979504, -0.2379501332623615, -0.1235896776596082, -0.05076228552112846, -0.09860504790827407, -0.3245990920293459, -0.04032587126953215, 0.9542721252832257, -0.08758059060477007, 0.0457278747167743], 28.381640423407596]
    최다 출현 개체수: 644
 
====== 27th generation ends ======
[[268, -0.725903366751484, 2.810504931871055, -0.03799830216275648, -0.2620096056101215, -0.11457291652152216, -0.1164266466089497, -0.1467944004734327, -0.17768846139105446, 0.03976078809836165, 0.9602392119016383, -0.10103241159346635, -0.04347725563869665], 24.510047818783992]
    최다

====== 7th generation ends ======
[[153, -1.0934092674861784, 3.1978103592452514, -0.136297150543224, 0.11608957536727381, 0.1865586152800818, 0.04441550736825029, -0.07596268266190355, -0.209205189107729, 0.6529363019843942, -0.08425598202130437, -0.396743870638311, -0.09753512502752797], 6.594600370421789]
    최다 출현 개체수: 1
 
====== 8th generation ends ======
[[216, -1.5530649721898437, 2.7727104826173505, -0.06461766749749849, 0.21730466186785716, 0.1385654787922281, 0.03700187170159927, 0.06351754197308403, -0.3138755460010686, 0.5436104456652314, -0.20909450458982312, -0.0860294910382054, -0.32638279087340455], 7.020886658830679]
    최다 출현 개체수: 1
 
====== 9th generation ends ======
[[245, -1.6693751224520859, 3.026861746589192, 0.16092543284836572, 0.2996352618202932, 0.039353594199954656, 0.05352450746009606, 0.05426457323794574, -0.5260632263410737, 0.38133995927102504, 0.010956671162319312, -0.2269665069539232, -0.24697026670500308], 7.352739502205491]
    최다 출현 개체수: 1
 
====== 

====== 32th generation ends ======
[[224, -2.0153655957500654, 2.771333049891604, -0.00964156639352842, 0.34066355129371934, 0.06181635785780627, 0.018340592305585452, -0.02660102374167826, -0.2290104626312334, 0.5791794985428889, -0.3660259129538933, -0.07809953538464012, -0.2906214988950266], 7.07035398064428]
    최다 출현 개체수: 611
 
====== 33th generation ends ======
[[223, -1.8153803314600765, 2.8765628827998753, -0.037101881742399145, 0.37075894090905337, 0.07372045291747074, -0.008166713072512193, 0.042053657846006155, -0.2882808760109537, 0.5134669483274698, -0.2751168622476738, -0.08841283068382996, -0.30292083624263116], 7.714691668807375]
    최다 출현 개체수: 28
 
====== 34th generation ends ======
[[219, -1.390361554037697, 2.933889215011068, -0.0363510534315906, 0.3362164792540886, 0.018837732217287642, 0.03173201306127191, 0.019395956112045032, -0.21037481310265094, 0.5938178193553069, -0.31241548853278306, -0.11926173125544062, -0.3215969136775348], 6.679309173039193]
    최다 출현 개체

====== 20th generation ends ======
[[210, -1.6005944313640952, 3.0548557936578304, 0.4641365264724126, -0.049075817184040846, 0.014814454588952551, -0.1246583743645489, -0.4917566660651879, 0.2965332973359863, 0.004177278630855178, 0.10684804021037202, 0.11349040276142129, -0.33450914238622237], 15.09437394815081]
    최다 출현 개체수: 699
 
====== 21th generation ends ======
[[218, -1.4108484021788366, 3.0108093710431625, 0.44562332969484597, -0.03628011227781317, 0.014677807443141606, -0.15186326653179122, -0.49367868042378477, 0.2945508068371286, 0.013243945737361212, 0.08679838165236822, 0.14510572863515436, -0.3181779407666109], 15.108380862151682]
    최다 출현 개체수: 748
  유전 알고리즘 종료
 
섹터: 비금속광물
 
  Train set: 20100129 ~ 20191129
  Test set: 20191129 ~ 20230927
 
  유전 알고리즘 시작
====== 0th generation ends ======
[[22, -2.0727349591899946, 2.2898741313643654, 0.2880378006553528, 0.20628426824214027, 0.505677931102507, -1.0], 9.873134018861592]
    최다 출현 개체수: 13
 
====== 1th generation ends =====

...

====== 8th generation ends ======
[[270, -1.801817038216602, 2.370826833778321, -0.019380173571064023, 0.23876932292907752, -0.007174198389599327, -0.34720614068479194, -0.0500412484240971, -0.5608018756219036, 0.10005875742738471, -0.015396363308543968, 0.38971103089170017, 0.27146088875183744], 7.708118218721419]
    최다 출현 개체수: 6
 
====== 9th generation ends ======
[[192, -1.7005538486463185, 1.9679227717782566, 0.0028729330582139777, 0.12018422313044172, 0.17834416059618893, -0.33786811327997496, -0.003863465252930744, -0.6237371849680209, 0.07191360875805786, -0.034531236499073345, 0.4655725204624705, 0.16111255399462712], 8.045493459765273]
    최다 출현 개체수: 14
 
====== 10th generation ends ======
[[225, -1.4547436707834083, 2.0317725746786355, 0.01223547183657523, 0.12177096243326976, -0.02493937804374561, -0.18893315259387558, -0.05408808384790586, -0.7088284230763373, 0.07973364054407284, -0.02321096243813563, 0.5933310664386545, 0.19292885874742755], 8.661211407094052]
    최다 출현 

====== 18th generation ends ======
[[227, -1.6316753179936567, 2.8825707322689214, -0.09163336707230961, 0.47024255684960165, -0.07540164944427115, 0.0029177076289413004, -0.6817844064605191, -0.09224556304293277, -0.01741612726363606, 0.4680582047307382, 0.058781530790718794, -0.04151888671633145], 14.994260022253597]
    최다 출현 개체수: 1
 
====== 19th generation ends ======
[[168, -0.5626922852090042, 2.8884594316196344, -0.01808642796090651, 0.05215611118244329, 0.1381702052679942, 0.11886946649779592, -0.6693257074645285, -0.3125878645745651, 0.15177199295013077, 0.49199354760909286, 0.04028740371323042, 0.006751272779312568], 14.265272128551139]
    최다 출현 개체수: 1
 
====== 20th generation ends ======
[[225, -1.6050837941978848, 2.8892687657635387, -0.05528756785810944, 0.07006074922372707, 0.20547774099664143, 0.03204602553647514, -0.644536033213897, -0.2916893192063449, 0.1078095113480683, 0.5733528775297981, 0.011253095365290003, -0.00848707972164856], 14.034722516044956]
    최다 출현 개체

====== 12th generation ends ======
[[147, -0.9322335027555861, 1.9744407430060957, 0.04362464308265241, 0.5556938415383597, 0.06886856053645567, -1.0, 0.3318129548425323], 7.138828392210422]
    최다 출현 개체수: 683
 
====== 13th generation ends ======
[[142, -1.0834088161112654, 1.8866976142019225, 0.0386553280904925, 0.5384292076103275, 0.0567244994811805, -1.0, 0.36619096481799934], 7.111276430910921]
    최다 출현 개체수: 771
  유전 알고리즘 종료
 
섹터: 운수창고업
 
  Train set: 20100129 ~ 20191129
  Test set: 20191129 ~ 20230927
 
  유전 알고리즘 시작
====== 0th generation ends ======
[[70, -1.1555668909430223, 3.5429285762323848, -0.2325361494744615, 1.0, -0.3447239787807618, -0.4227398717447766], 6.218038920932654]
    최다 출현 개체수: 22
 
====== 1th generation ends ======
[[68, -1.5777989983361198, 3.4069234765771985, -0.060242733561878814, 1.0, -0.526818205492801, -0.41293906094532007], 8.01126721152022]
    최다 출현 개체수: 8
 
====== 2th generation ends ======
[[21, -0.38093990477678086, 2.1410747573242817, 0.1733888081

====== 5th generation ends ======
[[65, -0.7566842671268819, 3.2381400021851188, -0.5316029583572031, 0.012047985353573355, -0.4683970416427969, 0.9879520146464267], 9.621755962658012]
    최다 출현 개체수: 28
 
====== 6th generation ends ======
[[68, -1.1983251036668792, 3.198516486624305, -0.8703849912213867, -0.011117587508438437, -0.11849742127017485, 1.0], 10.553844926428333]
    최다 출현 개체수: 32
 
====== 7th generation ends ======
[[59, -0.9675490199519164, 3.1753017511715584, -0.7289410193907121, 0.09809802773249361, -0.27105898060928785, 0.9019019722675063], 10.087723461907736]
    최다 출현 개체수: 103
 
====== 8th generation ends ======
[[65, -1.4352876712621112, 3.1695262675320555, -0.8286891104823908, 0.0313042566163555, -0.1713108895176092, 0.9686957433836445], 10.681239221012923]
    최다 출현 개체수: 72
 
====== 9th generation ends ======
[[63, -0.994291972196619, 3.0554289966506305, -0.6713861054929277, -0.01392754210218702, -0.31468635240488513, 1.0], 12.118333404287952]
    최다 출현 개체수: 171
 


====== 12th generation ends ======
[[139, -1.0388165109705938, 2.044073175356039, 0.1521617412468043, 0.05418303816737684, 0.29172444718154983, 0.021902531766469467, -0.008107188514076913, 0.015544312073991862, 0.07198832421274333, 0.3906300171493222, 0.0018655882017421124, -0.9918928114859231], 15.161336342043702]
    최다 출현 개체수: 217
 
====== 13th generation ends ======
[[123, -1.0215870928114443, 2.0312542183328266, 0.23989058110459485, 0.008773827880924937, 0.29011173527224277, 0.018159995949834265, 0.02422604405730743, 0.021280983096176955, 0.035030028567129126, 0.36252680407178955, -0.00412330616746578, -0.9958766938325342], 17.572321551428523]
    최다 출현 개체수: 628
 
====== 14th generation ends ======
[[139, -1.0225313436188004, 2.235064800217036, 0.19967336422452328, 0.03348584028816042, 0.25570060529823274, 0.022700385560174747, 0.03507708573053562, 0.01855789193691449, 0.014067934135069324, 0.4207368928263893, -0.0179738564737204, -0.9820261435262796], 17.71695694290333]
    최다 출현

#### 결과 정리

In [40]:
result_train = {}
result_test = {}
list_summary = []

for sector_code in sector_code_list:
    
    if len(dict_stk[sector_code]) == 0:
        continue
        
    params = optimized_params[sector_code]
    df_train = dict_train[sector_code]
    df_test = dict_test[sector_code]
    
    temp_train = simulation_func(df_train, params)
    temp_test = simulation_func(df_test, params)
    
    ret_train = pd.DataFrame([temp_train[1], temp_train[2], temp_train[3],temp_train[4],temp_train[5]], 
                             index = ['init_dt','exit_dt','ls','long','short']).T
    ret_test = pd.DataFrame([temp_test[1], temp_test[2], temp_test[3],temp_test[4],temp_test[5]], 
                            index = ['init_dt','exit_dt','ls','long','short']).T
    
    ret_train['init_dt'] = pd.to_datetime([str(int(x)) for x in ret_train['init_dt']])
    ret_train['exit_dt'] = pd.to_datetime([str(int(x)) for x in ret_train['exit_dt']])
    ret_test['init_dt'] = pd.to_datetime([str(int(x)) for x in ret_test['init_dt']])
    ret_test['exit_dt'] = pd.to_datetime([str(int(x)) for x in ret_test['exit_dt']])
    
    ret_train['holding period'] = ret_train['exit_dt'] - ret_train['init_dt']
    ret_test['holding period'] = ret_test['exit_dt'] - ret_test['init_dt']
    
    result_train[sector_code] = ret_train
    result_test[sector_code] = ret_test
    
    
    case_train = len(ret_train)
    
    if case_train == 0:
        hr_train = 0
    else:
        hr_train = len(ret_train[ret_train['ls'] >= 0])/len(ret_train)
        
    cumret_train = temp_train[0]
    
    case_test = len(ret_test)
    
    if case_test == 0:
        hr_test = 0
    else:
        hr_test = len(ret_test[ret_test['ls'] >= 0])/len(ret_test)
    
    cumret_test = temp_test[0]
                   
    
    list_summary.append([case_train, hr_train, cumret_train, case_test, hr_test, cumret_test])
    

In [44]:
df_summary = pd.DataFrame(list_summary, columns = ['cnt_train','hr_train','ret_train','cnt_test','hr_test','ret_test'], 
                          index = [dict_sector_name[x] for x in result_train.keys()])
df_summary['keys'] = result_train.keys()

In [45]:
df_summary

,cnt_train,hr_train,ret_train,cnt_test,hr_test,ret_test,keys
음식료품,2,1.000000,23.014734,0,0.000000,1.000000,IKS005
섬유의복,1,1.000000,15.342328,1,1.000000,1.081481,IKS006
화학,2,0.500000,6.724314,0,0.000000,1.000000,IKS008
의약품,3,1.000000,15.108381,0,0.000000,1.000000,IKS009
비금속광물,5,1.000000,16.251485,0,0.000000,1.000000,IKS010
철강금속,6,1.000000,4.863068,1,0.000000,0.730227,IKS011
기계,18,0.833333,34.862066,6,0.666667,-2.699022,IKS012
전기전자,8,1.000000,11.298945,2,1.000000,1.320821,IKS013
운수장비,6,1.000000,9.474732,0,0.000000,1.000000,IKS015
유통업,2,1.000000,15.262218,2,1.000000,1.382372,IKS016


In [51]:
result_train['IKS025']

,init_dt,exit_dt,ls,long,short,holding period
0,2010-03-03,2010-03-22,0.027288,-0.029186,0.056474,19 days
1,2010-04-13,2010-04-29,0.067928,0.123613,-0.055685,16 days
2,2010-05-19,2010-05-26,0.075624,-0.025185,0.100810,7 days
3,2010-06-25,2010-07-05,0.065361,0.055405,0.009955,10 days
4,2010-07-30,2010-08-31,0.014185,-0.082880,0.097066,32 days
...,...,...,...,...,...,...
79,2019-02-21,2019-02-26,0.028339,0.018031,0.010309,5 days
80,2019-03-11,2019-05-03,-0.036129,-0.012894,-0.023235,53 days
81,2019-07-02,2019-07-30,-0.013741,-0.041739,0.027998,28 days
82,2019-08-20,2019-09-06,0.035782,0.035069,0.000713,17 days


In [52]:
result_test['IKS025']

,init_dt,exit_dt,ls,long,short,holding period
0,2019-12-30,2020-02-05,-0.003012,-0.154876,0.151863,37 days
1,2020-03-05,2020-03-30,-0.030756,-0.181595,0.150839,25 days
2,2020-06-03,2020-06-15,0.080120,-0.038015,0.118135,12 days
3,2020-06-17,2020-07-01,0.058964,0.002250,0.056715,14 days
4,2020-08-20,2020-09-18,-0.021233,0.013304,-0.034537,29 days
5,2020-10-16,2020-12-10,-0.018652,-0.007874,-0.010778,55 days
6,2021-01-20,2021-02-17,0.013038,-0.027194,0.040232,28 days
7,2021-03-12,2021-03-16,0.043289,0.037166,0.006123,4 days
8,2021-04-16,2021-04-30,0.045002,0.077008,-0.032006,14 days
9,2021-06-08,2021-06-25,0.035226,0.061024,-0.025797,17 days


#### 병렬처리

In [79]:
ray.init(num_cpus=8)

2023-12-06 10:52:29,592	INFO worker.py:1664 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.8.8
Ray version:,2.8.0
Dashboard:,http://127.0.0.1:8265


In [80]:
filtered_sector = []
for sector_code in dict_stk.keys():
    if len(dict_stk[sector_code]) < 2:
        pass
    else:
        filtered_sector.append(sector_code)

In [81]:
dict_train_2 = {k:dict_train[k] for k in filtered_sector if k in dict_train}
dict_test_2 = {k:dict_test[k] for k in filtered_sector if k in dict_test}
dict_stk_2 = {k:dict_stk[k] for k in filtered_sector if k in dict_stk}
dict_sector_name_2 = {k:dict_sector_name[k] for k in filtered_sector if k in dict_sector_name}

In [82]:
@ray.remote
def get_optimized_params(sector_code, dict_train, dict_test, dict_stk, dict_sector_name):
    
    print(f'섹터: {dict_sector_name[sector_code]}')
    #print(' ')
    
    df_train = dict_train[sector_code]
    df_test = dict_test[sector_code]
    stk_list = dict_stk[sector_code]
    
    
    #print(f'  Train set: {train_begin_dt} ~ {train_end_dt}')
    #print(f'  Test set: {train_end_dt} ~ {max(df_test.index)}')
    #print(' ')
    #print(f'  유전 알고리즘 시작')
    n_generation = 200
    population = 1000
    best_sample = int(population/4)
    lucky_few = int(population/4)
    prob1 = 0.01
    #prob2 = 0.01
    alpha = 0.10
    r_param = 4
    param_length = len(stk_list)

    pop = generate_pop(df_train, size = population, length = param_length)

    best_gene = []
    best_perf = []

    g = 0

    while True:

        pop_sorted = compute_performance(df_train, pop)

        survivors = select_survivors(pop_sorted, best_sample, lucky_few, param_length, df_train)

        children = create_children(df_train, survivors, population, alpha, r_param)

        new_generation = mutate_pop(children, prob1)

        if len(new_generation) < population:
            #print(f'  세대수 부족: {len(new_generation)}')
            break

        fixed_generation = fix_1(new_generation, df_train, param_length)

        pop = fixed_generation
        best_gene.append(pop_sorted[0][0])
        best_perf.append(pop_sorted[0][1])


        #print(f'====== {g}th generation ends ======')
        print(pop_sorted[0])


        pop_wt = [x[0][3:] for x in pop_sorted]
        dist_wt = [sum([(x-y)**2 for (x,y) in zip(pop_wt[0],z)]) for z in pop_wt]
        hurdle = 0.025
        dist_wt_hurdle = [1 for x in dist_wt if x < hurdle]

        #print(f'    최다 출현 개체수: {sum(dist_wt_hurdle)}')
        if sum(dist_wt_hurdle) > population * 0.7:
            break

        g += 1

        if g > n_generation:
            break

        print(' ')

    #print(f'  유전 알고리즘 종료')
    #print(' ')
    
    return pop_sorted[0][0], g

In [83]:
futures = [get_optimized_params.remote(sector_code, dict_train_2, dict_test_2, dict_stk_2, dict_sector_name_2) for sector_code in filtered_sector]

(get_optimized_params pid=21128) 섹터: 음식료품
(get_optimized_params pid=15808) 섹터: 섬유의복
(get_optimized_params pid=30988) 섹터: 철강금속
(get_optimized_params pid=31764) 섹터: 의약품
(get_optimized_params pid=20192) 섹터: 기계
(get_optimized_params pid=21100) 섹터: 비금속광물
(get_optimized_params pid=25584) 섹터: 전기전자
(get_optimized_params pid=19320) 섹터: 화학


In [85]:
t1 = datetime.datetime.today()
optimized_params = ray.get(futures)

t2 = datetime.datetime.today()

lt = (t2 - t1).seconds

(get_optimized_params pid=19320) [[154, -1.5414387687635984, 3.121944692455533, -0.02625450342969824, -0.1306188485031018, -0.20472161118305485, 1.0, -0.174836256276912, -0.207971920284389, -0.011033389528153803, -0.1939164875276597, -0.04336662570843671, -0.007280357558594027], 5.858937344619739]
(get_optimized_params pid=19320)  
(get_optimized_params pid=19320)   [repeated 16x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(get_optimized_params pid=15808)   [repeated 16x across cluster]
(get_optimized_params pid=15808)   [repeated 16x across cluster]
(get_optimized_params pid=21128)   [repeated 10x across cluster]
(get_optimized_params pid=15808)   [repeated 6x across cluster]
(get_optimized_params pid=31764)   [repeated 6x across cluster]
(get_optimized_params pid=30988)   [repeated 6x across cluster]
(get_optimized_para

(get_optimized_params pid=31764)   [repeated 2x across cluster]
(get_optimized_params pid=21128) 섹터: 통신업
(get_optimized_params pid=19320)   [repeated 3x across cluster]
(get_optimized_params pid=30988)   [repeated 6x across cluster]
(get_optimized_params pid=31764)   [repeated 6x across cluster]
(get_optimized_params pid=19320)   [repeated 4x across cluster]
(get_optimized_params pid=21100)   [repeated 2x across cluster]
(get_optimized_params pid=15808) 섹터: 증권
(get_optimized_params pid=25584)   [repeated 7x across cluster]
(get_optimized_params pid=31764)   [repeated 2x across cluster]
(get_optimized_params pid=21100)   [repeated 6x across cluster]
(get_optimized_params pid=25584) [[158, -0.8159557844818953, 2.35765050080767, 0.09742889979892519, 0.6523094157814882, -0.03553273884458284, -0.3815837162598124, 0.08561174565030868, -0.14503792401659069, -0.03719940504135613, -0.3826035053011743, 0.16464993876927803, -0.018042710536483646], 12.937681625258616] [repeated 7x across cluster]


In [86]:
lt/60

27.0

In [88]:
optimized_params_1 = dict(zip(filtered_sector,[x[0] for x in optimized_params]))
optimized_params_2 = dict(zip(filtered_sector,[x[1] for x in optimized_params]))

In [89]:
result_train = {}
result_test = {}
list_summary = []

for sector_code in filtered_sector:
    
    if len(dict_stk[sector_code]) == 0:
        continue
        
    params = optimized_params_1[sector_code]
    df_train = dict_train[sector_code]
    df_test = dict_test[sector_code]
    
    temp_train = simulation_func(df_train, params)
    temp_test = simulation_func(df_test, params)
    
    ret_train = pd.DataFrame([temp_train[1], temp_train[2], temp_train[3],temp_train[4],temp_train[5]], 
                             index = ['init_dt','exit_dt','ls','long','short']).T
    ret_test = pd.DataFrame([temp_test[1], temp_test[2], temp_test[3],temp_test[4],temp_test[5]], 
                            index = ['init_dt','exit_dt','ls','long','short']).T
    
    ret_train['init_dt'] = pd.to_datetime([str(int(x)) for x in ret_train['init_dt']])
    ret_train['exit_dt'] = pd.to_datetime([str(int(x)) for x in ret_train['exit_dt']])
    ret_test['init_dt'] = pd.to_datetime([str(int(x)) for x in ret_test['init_dt']])
    ret_test['exit_dt'] = pd.to_datetime([str(int(x)) for x in ret_test['exit_dt']])
    
    ret_train['holding period'] = ret_train['exit_dt'] - ret_train['init_dt']
    ret_test['holding period'] = ret_test['exit_dt'] - ret_test['init_dt']
    
    result_train[sector_code] = ret_train
    result_test[sector_code] = ret_test
    
    
    case_train = len(ret_train)
    
    if case_train == 0:
        hr_train = 0
    else:
        hr_train = len(ret_train[ret_train['ls'] >= 0])/len(ret_train)
        
    cumret_train = temp_train[0]
    
    case_test = len(ret_test)
    
    if case_test == 0:
        hr_test = 0
    else:
        hr_test = len(ret_test[ret_test['ls'] >= 0])/len(ret_test)
    
    cumret_test = temp_test[0]
                   
    
    list_summary.append([case_train, hr_train, cumret_train, case_test, hr_test, cumret_test])
    

In [90]:
df_summary = pd.DataFrame(list_summary, columns = ['cnt_train','hr_train','ret_train','cnt_test','hr_test','ret_test'], 
                          index = [dict_sector_name[x] for x in result_train.keys()])
df_summary['keys'] = result_train.keys()

In [91]:
df_summary

,cnt_train,hr_train,ret_train,cnt_test,hr_test,ret_test,keys
음식료품,3,1.000000,21.913620,0,0.000000,1.000000,IKS005
섬유의복,1,1.000000,15.465863,1,1.000000,1.045376,IKS006
화학,2,1.000000,6.237558,0,0.000000,1.000000,IKS008
의약품,3,1.000000,10.062412,1,1.000000,1.223949,IKS009
비금속광물,5,1.000000,15.891223,0,0.000000,1.000000,IKS010
철강금속,8,1.000000,6.419614,1,1.000000,1.068145,IKS011
기계,23,0.956522,72.745922,5,0.400000,-3.473752,IKS012
전기전자,11,1.000000,12.937682,1,1.000000,1.179928,IKS013
운수장비,6,1.000000,9.090426,0,0.000000,1.000000,IKS015
유통업,3,1.000000,17.959750,0,0.000000,1.000000,IKS016


In [93]:
result_train['IKS012']

,init_dt,exit_dt,ls,long,short,holding period
0,2010-04-12,2010-04-29,0.106486,0.011962,0.094524,17 days
1,2010-06-22,2010-12-28,0.614451,0.702836,-0.088385,189 days
2,2011-01-27,2011-04-18,0.540992,0.316179,0.224813,81 days
3,2011-05-17,2011-08-18,0.429919,0.293762,0.136157,93 days
4,2011-10-06,2012-07-05,0.066227,0.205804,-0.139576,273 days
5,2012-07-27,2012-11-12,0.216415,-0.056557,0.272972,108 days
6,2013-01-03,2013-04-15,0.354080,0.164229,0.189851,102 days
7,2013-05-15,2013-08-21,0.111286,0.088631,0.022654,98 days
8,2013-10-07,2013-11-05,0.103223,-0.019723,0.122946,29 days
9,2014-01-15,2014-03-13,0.176052,0.129076,0.046976,57 days


In [94]:
result_test['IKS012']

,init_dt,exit_dt,ls,long,short,holding period
0,2020-01-23,2020-03-10,0.166819,-0.089049,0.255868,47 days
1,2020-04-13,2021-08-03,-3.733238,0.704423,-4.437661,477 days
2,2021-10-13,2022-02-14,-0.002537,-0.243913,0.241376,124 days
3,2022-02-28,2023-05-03,-0.014643,-0.177756,0.163113,429 days
4,2023-06-08,2023-08-10,0.108224,0.051478,0.056745,63 days
